In [1]:
import json
import datetime
import requests
import pandas as pd

# 팍스넷에서 데이터 주어옴. http://www.paxnet.co.kr/stock/analysis/chart?abbrSymbol=114800

# stock_no = "226490"
# file_name = "data_kodex_kospi"
# rename_prefix = "KDX"

stock_no = "114800"
file_name = "data_kodex_kospi_inverse"
rename_prefix = "KDI"

url = "http://www.paxnet.co.kr/stock/analysis/chart?abbrSymbol={}".format(stock_no)
res = requests.get(url)
result = res.text.split("var sise60minJson = \'")[1].split("\';\r\n \t")[0]
json_res = json.loads(result)
df_src = pd.DataFrame(json_res["body"]["chartDataList"])

In [2]:
df = df_src.copy()
date_list = [str(datetime.datetime.fromtimestamp(x)).split(" ")[0] for x in df["tradeDt"]]
time_list = [str(datetime.datetime.fromtimestamp(x)).split(" ")[1].split(":")[0] for x in df["tradeDt"]]
df["date"] = date_list
df["time"] = time_list
df["datetime"] = df["date"].str.replace("-", "") + df["time"]
df = df[['date', 'time', 'datetime', 'openPrice', 'closePrice', 'highPrice', 'lowPrice', 'volume']]
df.rename(columns={
    "date" : "DATE",
    "time" : "TIME",
    "datetime" : "DATETIME",    
    "openPrice" : "{}_OPN".format(rename_prefix),
    "closePrice" : "{}_CLS".format(rename_prefix),
    "highPrice" : "{}_HIGH".format(rename_prefix),
    "lowPrice" : "{}_LOW".format(rename_prefix),
    "volume" : "{}_VOL".format(rename_prefix)
}, inplace=True)

df_old = pd.read_excel("{}.xlsx".format(file_name))
df_new = pd.concat([df_old, df], axis=0, ignore_index=True)
df_new["DATETIME_INT"] = df_new["DATETIME"].astype("int")
df_new.drop_duplicates(["DATETIME_INT"], inplace=True)
df_new.drop(columns=["DATETIME_INT"], inplace=True)
df_new.reset_index(drop=True, inplace=True)
df_new.to_excel("{}.xlsx".format(file_name), index=False)
